# Applied Data Science Capstone Project
## Final Project of [IBM's Data Science Professional Certificate Course] (https://www.coursera.org/professional-certificates/ibm-data-science)

First, let's import all the libraries needed

In [1]:
import pandas as pd
import numpy as np 

from pandas.io.html import read_html

### Downloading Data
Let's scrape the Toronto's neighborhoods dataframe from a wikipedia table

In [2]:
# Get a list of wiki tables from the following link 
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitable = read_html(page,  attrs = {"class":"wikitable"})

# Get the dataframe for the first table 
df_toronto = wikitable[0]

### Pre-processing

In [3]:
# Drop all rows where borough is not assigned
df_toronto.drop(df_toronto[df_toronto['Borough'] == 'Not assigned'].index, inplace = True)

# As neighborhoods are already grouped by postal codes, only replace the slashes with commas 
for i in df_toronto.index:
    df_toronto.at[i, 'Neighborhood'] = df_toronto.at[i, 'Neighborhood'].replace(" /", ",")

# Also, there are no "Not assigned" neighborhoods, so there is no need to correct it

The shape of the pre-processed dataframe is:

In [4]:
df_toronto.shape

(103, 3)